##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [25]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow as tf

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

Xception=tf.keras.applications.Xception(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = 'imagenet')

print(Xception.summary())


pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

for layer in Xception.layers:
  layer.trainable= False
  
#pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


Xception_last_layer=Xception.get_layer('block14_sepconv2_act')
Xception_last_output=Xception_last_layer.output

--2020-05-14 09:15:40--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 2404:6800:4008:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  97.8MB/s    in 0.9s    

2020-05-14 09:15:41 (97.8 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

83689472/83683744 [==============================] - 2s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)    

In [0]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(Xception_last_output)
#x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

#model = Model( pre_trained_model.input, x)
model2=Model(Xception.input,x) 

#model.compile(optimizer = RMSprop(lr=0.0001), 
              #loss = 'binary_crossentropy', 
              #metrics = ['accuracy'])

model2.compile(optimizer='adam',loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


In [28]:
!wget --no-check-certificate \
        https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
       -O /tmp/cats_and_dogs_filtered.zip

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

--2020-05-14 09:16:12--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.23.128, 2404:6800:4008:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.23.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   100MB/s    in 0.7s    

2020-05-14 09:16:13 (100 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [29]:
history2 = model2.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

Epoch 1/20
100/100 - 26s - loss: 2.7558 - accuracy: 0.8805 - val_loss: 0.4779 - val_accuracy: 0.9530
Epoch 2/20
100/100 - 25s - loss: 0.7097 - accuracy: 0.9080 - val_loss: 0.1645 - val_accuracy: 0.9570
Epoch 3/20
100/100 - 25s - loss: 0.2377 - accuracy: 0.9150 - val_loss: 0.0973 - val_accuracy: 0.9590
Epoch 4/20
100/100 - 25s - loss: 0.1563 - accuracy: 0.9405 - val_loss: 0.1045 - val_accuracy: 0.9580
Epoch 5/20
100/100 - 25s - loss: 0.1890 - accuracy: 0.9330 - val_loss: 0.1072 - val_accuracy: 0.9640
Epoch 6/20
100/100 - 25s - loss: 0.1630 - accuracy: 0.9305 - val_loss: 0.1164 - val_accuracy: 0.9580
Epoch 7/20
100/100 - 25s - loss: 0.1480 - accuracy: 0.9395 - val_loss: 0.0916 - val_accuracy: 0.9620
Epoch 8/20
100/100 - 25s - loss: 0.1558 - accuracy: 0.9400 - val_loss: 0.1162 - val_accuracy: 0.9560
Epoch 9/20
100/100 - 25s - loss: 0.1236 - accuracy: 0.9530 - val_loss: 0.0728 - val_accuracy: 0.9760
Epoch 10/20
100/100 - 25s - loss: 0.1265 - accuracy: 0.9510 - val_loss: 0.0999 - val_accura

In [0]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)




100/100 - 22s - loss: 0.2135 - accuracy: 0.9455 - val_loss: 0.2408 - val_accuracy: 0.9540 -InceptionV3

100/100 - 25s - loss: 0.1453 - accuracy: 0.9455 - val_loss: 0.0854 - val_accuracy: 0.9670 -Xception


In [0]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()